In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import panel as pn

pn.extension('tabulator')
import hvplot.pandas

In [3]:
df1 = pd.read_csv('deaths-by-cause.csv')
df1.head(2)

,Entity,Code,Year,Number of executions (Amnesty International),Deaths - Meningitis - Sex: Both - Age: All Ages (Number),Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number),Deaths - Parkinson's disease - Sex: Both - Age: All Ages (Number),Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number),Deaths - Malaria - Sex: Both - Age: All Ages (Number),Deaths - Drowning - Sex: Both - Age: All Ages (Number),...,Deaths - Chronic kidney disease - Sex: Both - Age: All Ages (Number),Deaths - Poisonings - Sex: Both - Age: All Ages (Number),Deaths - Protein-energy malnutrition - Sex: Both - Age: All Ages (Number),Terrorism (deaths),Deaths - Road injuries - Sex: Both - Age: All Ages (Number),Deaths - Chronic respiratory diseases - Sex: Both - Age: All Ages (Number),Deaths - Cirrhosis and other chronic liver diseases - Sex: Both - Age: All Ages (Number),Deaths - Digestive diseases - Sex: Both - Age: All Ages (Number),"Deaths - Fire, heat, and hot substances - Sex: Both - Age: All Ages (Number)",Deaths - Acute hepatitis - Sex: Both - Age: All Ages (Number)
0,Afghanistan,AFG,2007,15,2933.0,1402.0,450.0,2488.0,393.0,2127.0,...,4490.0,512.0,2439.0,1199.0,7425.0,7222.0,3346.0,6458.0,481.0,3437.0
1,Afghanistan,AFG,2008,17,2731.0,1424.0,455.0,2277.0,255.0,1973.0,...,4534.0,495.0,2231.0,1092.0,7355.0,7143.0,3316.0,6408.0,462.0,3005.0


In [4]:
df2 = pd.read_csv('gdp-per-capita.csv')
df2.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,25496.84394,26442.426800,26895.05717,28399.050130,28453.715560,29348.418970,30253.714230,31135.884360,23384.298790,NaN
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,162.726326,162.555968,172.271022,199.784916,180.228774,199.517227,...,1777.30395,1748.905594,1736.24222,1556.316469,1446.533624,1629.404273,1541.031661,1511.309259,1360.878645,1557.722682


In [5]:
df2 = pd.melt(df2, id_vars=['Country Name', 'Country Code'], 
        value_vars=list(df2.columns[4:].values))
df2.columns = ['Entity', 'Code', 'Year', 'gdp_per_capita']

In [6]:
#import geojson file and create dataframe from extracted geojson data.
import json
wrld = json.load(open('world.geo.json', 'r'))
subregion = [country['properties']['subregion'] for country in wrld['features']]
name = [country['properties']['sovereignt'] for country in wrld['features']]
#economy = [country['properties']['economy'] for country in wrld['features']]
region = [country['properties']['region_wb'] for country in wrld['features']]
continent = [country['properties']['continent'] for country in wrld['features']]


zipped = list(zip(subregion, name, region, continent))   
country_deets = pd.DataFrame(zipped, columns = ['subregion', 'name', 'region', 'continent'])
country_deets.head(1)

,subregion,name,region,continent
0,Caribbean,The Bahamas,Latin America & Caribbean,North America


In [7]:
df2 = pd.merge(df2, country_deets, 
               left_on = ['Entity'], 
               right_on = ['name'], 
               how = "inner")
df2['Year'] = df2.Year.astype('int')

In [8]:
df1['total_deaths'] = df1.iloc[:, 3:].sum(axis=1)

C:\Users\adejimi\AppData\Local\Temp\ipykernel_31432\3420572590.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df1['total_deaths'] = df1.iloc[:, 3:].sum(axis=1)


In [9]:
# Add GDP per capita data to the deaths data
df = pd.merge(df1, df2, on = ['Entity', 'Year', 'Code'], how = "inner")

In [10]:
df.shape

(4290, 42)

In [11]:
df.columns = [x.split(' - ')[1] if ' - ' in x else  x.split(' - ')[0] for x in list(df.columns.values)]

In [12]:
df= df.fillna(0)
df.columns = ['Entity', 'Code', 'Year',
       'Number of executions (Amnesty International)', 'Meningitis',
       "Alzheimer's disease and other dementias", "Parkinson's disease",'Nutritional deficiencies', 'Malaria', 'Drowning',
       'Interpersonal violence', 'Maternal disorders', 'HIV/AIDS','Drug use disorders', 'Tuberculosis', 'Cardiovascular diseases',
       'Lower respiratory infections', 'Neonatal disorders','Alcohol use disorders', 'Self-harm',
       'Exposure to forces of nature', 'Diarrheal diseases', 'Environmental heat and cold exposure', 'Neoplasms',
       'Conflict and terrorism', 'Diabetes mellitus',
       'Chronic kidney disease', 'Poisonings','Protein-energy malnutrition', 'Terrorism (deaths)',
       'Road injuries', 'Chronic respiratory diseases','Cirrhosis and other chronic liver diseases', 'Digestive diseases',
       'Fire, heat, and hot substances', 'Acute hepatitis', 'total_deaths','gdp_per_capita', 'subregion', 'name', 'region', 'continent']

In [13]:
df = df.drop('Number of executions (Amnesty International)', axis=1)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4290 entries, 0 to 4289
Data columns (total 41 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Entity                                      4290 non-null   object 
 1   Code                                        4290 non-null   object 
 2   Year                                        4290 non-null   int64  
 3   Meningitis                                  4290 non-null   float64
 4   Alzheimer's disease and other dementias     4290 non-null   float64
 5   Parkinson's disease                         4290 non-null   float64
 6   Nutritional deficiencies                    4290 non-null   float64
 7   Malaria                                     4290 non-null   float64
 8   Drowning                                    4290 non-null   float64
 9   Interpersonal violence                      4290 non-null   float64
 10  Maternal dis

In [15]:
idf = df.interactive()

In [16]:
import datetime as dt
date_slider = pn.widgets.IntSlider(
                        name = 'Date Slider',
                        start = 1990,
                        end =2019,
                        step = 5,
                        value = 1995
)
date_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'30b847ea-931b-4863-a4ff-fe1b39e54c02': {'defs': …

In [17]:
yaxis_dth = pn.widgets.MultiSelect(name='Cause of Death', 
                        value=list(df.columns.values[3:-5]),
                        options=list(df.columns.values[3:-5]), 
                        size=8)

In [18]:
continents = ['World', 'Asia','Europe', 'Africa', 'North America', 'South America','Oceania']

cov_pipeline = (
    idf[
        (idf.Year <= date_slider) &
        (idf.continent.isin(continents))
    ]
    .groupby(['continent', 'Year'])[yaxis_dth].mean()
    #.to_frame()
    .reset_index()
    .sort_values(by = 'Year')
    .reset_index(drop=True)
)

In [19]:
cov_plot = cov_pipeline.hvplot(x = 'Year',
            by = 'continent',
            y = yaxis_dth,
            width =2,
            title = 'Annual deaths by Continent',
            kind = 'line')
cov_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'9fd32da0-0edd-4971-9d2a-5bd3513c7851': {'defs': …

In [20]:
cov_table = cov_pipeline.pipe(pn.widgets.Tabulator, 
                                pagination = 'remote', 
                                page_size=10,
                                sizing_mode = 'stretch_width')
cov_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'0f0db528-949e-4af0-bdb6-e508156d9e5d': {'defs': …

In [21]:
dth_gdp_pipeline = (
    idf[
        (idf.Year <= date_slider) &
        (idf.continent.isin(continents))
    ]
    .groupby(['continent', 'Year', 'gdp_per_capita'])['total_deaths'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'Year')
    .reset_index(drop=True)
)


In [22]:
dth_gdp_pipeline

BokehModel(combine_events=True, render_bundle={'docs_json': {'45267dfc-5a9b-458c-b867-14f5c22e1852': {'defs': …

In [23]:
dthgdp_scatter = dth_gdp_pipeline.hvplot(x = 'gdp_per_capita',
                                            y = 'total_deaths',
                                            by = 'continent',
                                            size = 80,
                                            kind = 'scatter',
                                            legend = True,
                                            height = 500,
                                            width = 800)
dthgdp_scatter

BokehModel(combine_events=True, render_bundle={'docs_json': {'627d50bb-9e68-4979-8f38-c127dbc889e8': {'defs': …

In [24]:
yaxis_bar_dth = pn.widgets.MultiSelect(name='Cause of Death', 
                        value=list(df.columns.values[3:-5]),
                        options=list(df.columns.values[3:-5]), 
                        size=8)

In [26]:
continent_cause_dth = cov_pipeline.hvplot.bar(x='continent', y=yaxis_dth, height=500)

In [32]:
dash_template = pn.template.FastListTemplate(
    title = 'Cause of Death Dashboard',
    sidebar= [pn.pane.Markdown('# Cause of Death'),
              pn.pane.Markdown("### Death, they say, is a constant part of life. Information about the most common cause of death can help prevent these causes and improve the life expectancy of many."),
              pn.pane.JPG('Death_and_Life.png', sizing_mode='scale_both'),
              pn.pane.Markdown('## Settings'),
              date_slider],
    main = [pn.Row(pn.Column(yaxis_dth, 
                             cov_plot.panel(width = 700), margin = (0, 25)),
                   cov_table.panel(width = 500)),
            pn.Row(pn.Column(dthgdp_scatter.panel(width = 600), margin=(0, 25)), 
                   pn.Column(yaxis_dth, continent_cause_dth.panel(width = 600)))],
    accent_base_color = '#88d8b0',
    header_background = '#88d8b0' 
)
#dash_template.show()
dash_template.servable()

ValueError: JPG pane cannot parse string that is not a filename or URL.

In [29]:
os.getcwd()

'c:\\Users\\adejimi\\OneDrive - ARO Volcani Center\\DA_Projects\\DTH'